In [1]:
import pandas as pd
import numpy as np


In [2]:
twm_accounts = pd.read_csv('data/twm_accounts.csv', delimiter=';')
twm_customer = pd.read_csv('data/twm_customer.csv', delimiter=';')
twm_transactions = pd.read_csv('data/twm_transactions.csv', delimiter=';')
twm_checking_acct = pd.read_csv('data/twm_checking_acct.csv', delimiter=';')
twm_checking_tran = pd.read_csv('data/twm_checking_tran.csv', delimiter=';')
twm_credit_acct = pd.read_csv('data/twm_credit_acct.csv', delimiter=';')
twm_credit_tran = pd.read_csv('data/twm_credit_tran.csv', delimiter=';')
twm_savings_acct = pd.read_csv('data/twm_savings_acct.csv', delimiter=';')
twm_savings_tran = pd.read_csv('data/twm_savings_tran.csv', delimiter=';')

twm_customer['state_code'] = twm_customer['state_code'].map(str.strip)
twm_customer['gender'] = twm_customer['gender'].map(str.strip)
twm_accounts['acct_type'] = twm_accounts['acct_type'].map(str.strip)
twm_accounts['account_active'] = twm_accounts['account_active'].map(str.strip)

twm_transactions['channel'] = twm_transactions['channel'].map(str.strip)

twm_checking_acct['account_active'] = twm_checking_acct['account_active'].map(str.strip)
twm_checking_tran['channel'] = twm_checking_tran['channel'].map(str.strip)
twm_checking_tran['tran_code'] = twm_checking_tran['tran_code'].map(str.strip)
twm_credit_acct['account_active'] = twm_credit_acct['account_active'].map(str.strip)
twm_credit_tran['tran_code'] = twm_credit_tran['tran_code'].map(str.strip)
twm_credit_tran['channel'] = twm_credit_tran['channel'].map(str.strip)
twm_savings_acct['acct_type'] = twm_savings_acct['acct_type'].map(str.strip)
twm_savings_tran['channel'] = twm_savings_tran['channel'].map(str.strip)
twm_savings_tran['tran_code'] = twm_savings_tran['tran_code'].map(str.strip)

AttributeError: module 'pandas' has no attribute 'to_csv'

In [7]:
def dummy_var_create(df, column_name):
    for name in set(df[column_name]):
        if str(name).strip() != "":
            df[f"{column_name}_dummy_{name}"] = (df[column_name] == name).astype(int)


twm_transactions2 = pd.DataFrame(twm_transactions)
# dummy_var_create(twm_transactions2, 'channel')
twm_transactions
twm_transactions2

,tran_id,acct_nbr,tran_amt,principal_amt,interest_amt,new_balance,tran_date,tran_time,channel,tran_code,channel_dummy_V,channel_dummy_E,channel_dummy_H,channel_dummy_B,channel_dummy_K,channel_dummy_M,channel_dummy_A,channel_dummy_P,channel_dummy_C,channel_dummy_T
0,27,13625623,0.00,0.00,0.0,3753.34,21.10.1995,121656,A,IQ,0,0,0,0,0,0,1,0,0,0
1,97,13628392,0.00,0.00,0.0,254.49,5.2.1995,153053,V,IQ,1,0,0,0,0,0,0,0,0,0
2,21,13630842,-97.57,-97.57,0.0,3819.56,23.7.1995,0,P,WD,0,0,0,0,0,0,0,1,0,0
3,44,13631412,-0.15,-0.15,0.0,224.05,30.1.1995,0,,FK,0,0,0,0,0,0,0,0,0,0
4,31,13625722,0.00,0.00,0.0,240.55,25.1.1995,204521,B,IQ,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77575,77,13631752,-95.71,-95.71,0.0,95.71,14.2.1995,161001,A,TR,0,0,0,0,0,0,1,0,0,0
77576,94,13626772,-93.90,-93.90,0.0,824.36,11.3.1995,0,P,WD,0,0,0,0,0,0,0,1,0,0
77577,42,13625262,-10.35,-10.35,0.0,10.36,26.8.1995,0,P,WD,0,0,0,0,0,0,0,1,0,0
77578,98,13627052,-423.80,-423.80,0.0,162.28,13.6.1995,110209,E,WD,0,1,0,0,0,0,0,0,0,0


In [ ]:
# twm_customer = pd.to_csv('data/twm_customer.csv', delimiter=',')
# twm_accounts = pd.to_csv('data/twm_accounts.csv', delimiter=',')
# twm_transactions = pd.to_csv('data/twm_transactions.csv', delimiter=',')
# twm_checking_acct =
#
#
# channel,trans_code